In [12]:
import os
import numpy as np
import pandas as pd
import cv2
import gc
from tqdm import tqdm
from glob import glob


In [13]:
dirs = os.listdir('data')
print(dirs)
images_path = []
labels = []
for folder in dirs:
    path = glob(f'./data/{folder}/*.jpg')
    label =['{}'.format(folder)]*len(path)
    # append
    images_path.extend(path)
    labels.extend(label)

['Mask', 'Mask_Chin', 'Mask_Mouth_Chin', 'Mask_Nose_Mouth']


In [14]:
len(labels)

10000

In [15]:
face_detection_model = cv2.dnn.readNetFromCaffe('./models/deploy.prototxt.txt',
                                                './models/res10_300x300_ssd_iter_140000_fp16.caffemodel')

In [6]:
def face_detection_dnn(img):
    # blob from image (rgb mean subraction image)
    image = img.copy()
    h,w = image.shape[:2]
    blob = cv2.dnn.blobFromImage(image,1,(300,300),(104,117,123),swapRB=True)
    # get the detections
    face_detection_model.setInput(blob)
    detections = face_detection_model.forward()
    for i in range(0,detections.shape[2]):
        confidence = detections[0,0,i,2] # confidence score
        if confidence > 0.5:
            box = detections[0,0,i,3:7]*np.array([w,h,w,h])
            box = box.astype(int)
            #print(box)
            pt1 = (box[0],box[1])
            pt2 = (box[2],box[3])
            #cv2.rectangle(image,pt1,pt2,(0,255,0),2)
            roi = image[box[1]:box[3],box[0]:box[2]]

            return roi
    return None

In [19]:
img_path = images_path[0]
img = cv2.imread(img_path)
face = face_detection_dnn(img)


cv2.imshow('face', face) # The face part of the image
cv2.imshow('original',img) # The entire image itself
cv2.waitKey()
cv2.destroyAllWindows()

In [20]:
def datapreprocess(img):
    # blob from image (rgb mean subtraction image)
    face = face_detection_dnn(img)
    if face is not None:

        # blob from image
        blob = cv2.dnn.blobFromImage(face,1,(100,100),(104,117,123),swapRB=True)
        blob_squeeze = np.squeeze(blob).T
        blob_rotate = cv2.rotate(blob_squeeze,cv2.ROTATE_90_CLOCKWISE)
        blob_flip = cv2.flip(blob_rotate,1)
        # remove negative values and normalize
        img_norm = np.maximum(blob_flip,0)/blob_flip.max()
    
        return img_norm
    else:
        return None



In [ ]:

data_img = []
label_img = []
i = 0
for path, label in tqdm(zip(images_path,labels),desc='preprocessing'):
    img = cv2.imread(path)
    process_img = datapreprocess(img)
    if process_img is not None:
        data_img.append(process_img)
        label_img.append(label)
          
    i += 1
    if i%100 == 0:
        gc.collect()

In [ ]:
X = np.array(data_img)
y = np.array(label_img)

In [ ]:
X.shape, y.shape

In [ ]:
np.savez('./data/data_preprocess.npz',X,y)